# Data Cleaning and Consolidating it into a single .csv file

NOTE: The below code is supposed to be run on a Jupyter notebook with the PySpark kernel on Spark EMR cluster

In [1]:
%%configure -f
{
    "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
        "spark.sql.execution.arrow.enabled": "true"
    }
}

In [2]:
sc.install_pypi_package("boto3==1.19.2")
sc.install_pypi_package("pandas==1.0.5")
sc.install_pypi_package("scipy==1.4.1")
sc.install_pypi_package("matplotlib==3.2.1")
sc.install_pypi_package("seaborn==0.10.1")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1685070668821_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import boto3

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
bucket = "chi-taxi"
input_path = "raw_csvs/"
# output_path_parq = "consolidated/consolidated_chi_taxi.parquet"
output_path_csv = "consolidated/consolidated_chi_taxi_csv"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
s3_client = boto3.client('s3')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
response = s3_client.list_objects_v2(Bucket=bucket, Prefix=input_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
csv_files = [obj['Key'] for obj in response['Contents'] if obj['Key'].endswith('.csv')]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
len(csv_files)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

156

This indicates that we have successfully read s3_key names of all the 156 data parts we have extracted and stored in S3

We will use Spark to read-in and consolidate these disparate csv file into a single csv which can be read-in via Dask for data visualization

In [10]:
merged_df = None

for csv_file in csv_files:
    try:
        df = spark.read.csv(f"s3://{bucket}/{csv_file}", header=True, inferSchema=True)
        if merged_df is None:
            merged_df = df
        else:
            merged_df = merged_df.union(df)
    except:
        print(f"Error at csv: {csv_file}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
merged_df.coalesce(1).write.csv(f"s3a://{bucket}/{output_path_csv}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

NOTE: We use coalescne here to ensure that we get one single consolidated csv instead of a csv directory with multiple parts